<a href="https://colab.research.google.com/github/UshaSupriya/ImportingDFtomysqlDB.ipynb/blob/main/ImportingDFtomysqlDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMySQL

In [ ]:
import pymysql
import pandas as pd

In [ ]:
pymysql.connect(host='hostname', user='username', passwd='pwd',db= 'dbname',port=3306)

In [ ]:
conn = pymysql.connect(host='hostname', user='username', passwd='pwd',db= 'dbname',port=3306)

In [ ]:
curr = conn.cursor()

In [ ]:
curr

In [ ]:
#upload your CSV files here
df = pd.read_csv('youtube_vids_info_pull.csv', index_col=0) 
df.head()

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,jN5hpgBrz8k,Dealing With Python Date Manipulations In a Me...,2022-03-22,3247,105,0,20
1,yFyyP_Uvxj0,How To Solve Data Science Probability Intervie...,2022-03-10,2025,62,0,2
2,Av92CaPTRr8,How to use SQL window functions: RANK vs DENSE...,2022-03-02,2620,76,0,9
3,VEjxlKBkZGM,Amazon Data Science Interview Question Walkthr...,2022-02-08,5707,179,0,20
4,hMUf7DqG1nQ,SQL Basics: How to Join Multiple Tables in SQL...,2022-01-27,3252,90,0,6


In [ ]:
df.shape

(85, 7)

In [ ]:
#Taken from github site
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    dislike_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
            )""")

    curr.execute(create_table_command)

def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        view_count, like_count, dislike_count,comment_count)
    VALUES(%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)


def update_row(curr, video_id, video_title, view_count, like_count, dislike_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                dislike_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, dislike_count, comment_count, video_id)
    curr.execute(query, vars_to_update)


def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM videos WHERE video_id = %s""")

    curr.execute(query, (video_id,))
    return curr.fetchone() is not None



def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)


def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count']
                          , row['like_count'], row['dislike_count'], row['comment_count'])


def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'dislike_count', 'comment_count'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['dislike_count'],row['comment_count'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df


In [ ]:
#create table

create_table(curr)

In [ ]:
#update data for existing videos

new_vid_df = update_db(curr,df)
conn.commit()

In [ ]:
new_vid_df.head()

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,jN5hpgBrz8k,Dealing With Python Date Manipulations In a Me...,2022-03-22,3247,105,0,20
1,yFyyP_Uvxj0,How To Solve Data Science Probability Intervie...,2022-03-10,2025,62,0,2
2,Av92CaPTRr8,How to use SQL window functions: RANK vs DENSE...,2022-03-02,2620,76,0,9
3,VEjxlKBkZGM,Amazon Data Science Interview Question Walkthr...,2022-02-08,5707,179,0,20
4,hMUf7DqG1nQ,SQL Basics: How to Join Multiple Tables in SQL...,2022-01-27,3252,90,0,6


In [ ]:
new_vid_df.shape

(85, 7)

In [ ]:
#insert new videos into db table

append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [ ]:
#view data in db table

curr.execute("SELECT * FROM videos")
print(curr.fetchall())

(('_gy1o9UH2dQ', 'Hard SQL Interview Question From FACEBOOK | Data Science Coding Interviews (Popularity Percentage)', datetime.date(2020, 11, 4), 21211, 452, 0, 71), ('-i9chjBSOqU', 'merge() Method to Join Multiple Pandas Dataframes - Beginner Python Pandas Tutorial #7', datetime.date(2020, 5, 29), 1478, 34, 0, 23), ('01kOJUiiEQE', 'NumPy Slicing and Accessing Subarrays - Complete Python NumPy Tutorial for Beginners #5', datetime.date(2020, 5, 14), 373, 6, 0, 0), ('0GpgMvyN0Fg', 'How I Learned Data Science (resources to get a job) in 2021', datetime.date(2021, 2, 4), 11899, 720, 0, 45), ('0hjTJw9Jpmc', 'How to Reshape NumPy Arrays - Beginner Python NumPy Exercises #5', datetime.date(2020, 5, 14), 313, 5, 0, 1), ('1SgHHQeRuxs', 'How I Prepare For My Data Science Interviews (From a Data Scientist)', datetime.date(2020, 10, 18), 4373, 131, 0, 17), ('2iE3JgNTwVU', 'A Walkthrough of Data Science SQL Interview Question from Noom (Date Manipulations)', datetime.date(2021, 6, 4), 4826, 158, 0

In [ ]:
#Now let's update rows. I have downloaded latest youtube_vids_info_pull.csv files which contains 86 videos - so now 1 row will be added and remaining 85 videos' rows will be updated with view/like/dislike count
#upload your CSV files here
df = pd.read_csv('youtube_vids_info_pull_latest.csv', index_col=0) 
df.head()

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,OzyHfddLj9g,How to Solve a SQL Interview Question Using WH...,2022-04-13,1900,68,0,7
1,jN5hpgBrz8k,Dealing With Python Date Manipulations In a Me...,2022-03-22,3891,125,0,20
2,yFyyP_Uvxj0,How To Solve Data Science Probability Intervie...,2022-03-10,2158,66,0,3
3,Av92CaPTRr8,How to use SQL window functions: RANK vs DENSE...,2022-03-02,3017,83,0,9
4,VEjxlKBkZGM,Amazon Data Science Interview Question Walkthr...,2022-02-08,6016,186,0,20


In [ ]:
df.shape

(86, 7)

In [ ]:
#update data for existing videos

new_vid_df2 = update_db(curr,df)
conn.commit()

In [ ]:
new_vid_df2

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,OzyHfddLj9g,How to Solve a SQL Interview Question Using WH...,2022-04-13,1900,68,0,7


In [ ]:
#insert new videos into db table

append_from_df_to_db(curr, new_vid_df2)
conn.commit()

In [ ]:
#view data in db table

curr.execute("SELECT * FROM videos")
print(curr.fetchall())

(('_gy1o9UH2dQ', 'Hard SQL Interview Question From FACEBOOK | Data Science Coding Interviews (Popularity Percentage)', datetime.date(2020, 11, 4), 21693, 465, 0, 71), ('-i9chjBSOqU', 'merge() Method to Join Multiple Pandas Dataframes - Beginner Python Pandas Tutorial #7', datetime.date(2020, 5, 29), 1530, 35, 0, 23), ('01kOJUiiEQE', 'NumPy Slicing and Accessing Subarrays - Complete Python NumPy Tutorial for Beginners #5', datetime.date(2020, 5, 14), 397, 8, 0, 0), ('0GpgMvyN0Fg', 'How I Learned Data Science (resources to get a job) in 2021', datetime.date(2021, 2, 4), 12236, 743, 0, 45), ('0hjTJw9Jpmc', 'How to Reshape NumPy Arrays - Beginner Python NumPy Exercises #5', datetime.date(2020, 5, 14), 324, 5, 0, 1), ('1SgHHQeRuxs', 'How I Prepare For My Data Science Interviews (From a Data Scientist)', datetime.date(2020, 10, 18), 4394, 132, 0, 17), ('2iE3JgNTwVU', 'A Walkthrough of Data Science SQL Interview Question from Noom (Date Manipulations)', datetime.date(2021, 6, 4), 4915, 159, 0

Note: Clearly the counts have changed after few weeks. So our data is being added and updated as expected